<a href="https://colab.research.google.com/github/tonykipkemboi/langchain-csv-agent-gpt-4o/blob/main/langchain_csv_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
This tutorial will guide you through the process of using LangChain agents to analyze CSV data.

*   We'll be using Google Colab as our
development environment, which provides a convenient platform for running Python code and installing necessary packages.

* LangChain is a powerful framework for developing applications powered by language models.

* In this tutorial, we'll focus on using LangChain's CSV agent, which allows us to interact with CSV data using natural language queries.

improved-horizontal-langchain-csv-agent-flowchart.svg

## Install dependencies

In [1]:
!pip install --q langchain langchain_experimental pandas langchain_google_vertexai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.21.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.3 which is incompatible.


## Import libraries and set up Gemini Google

In [2]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

FOLDER_NAME="."

In [3]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

from langchain_experimental.agents import create_csv_agent
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name="gemini-1.5-pro-001", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location=LOCATION, # Your Vertex AI location
)

# agent_executor = create_pandas_dataframe_agent(
#     llm,
#     "titanic.csv",
#     agent_type="openai-tools",
#     verbose=True
# )

## Upload CSV files

#### Consumer complaint database data

> The Consumer Complaint Database is a collection of complaints about consumer financial products and services. Complaints are published after the company responds, confirming a commercial relationship with the consumer, or after 15 days, whichever comes first. Complaints referred to other regulators, such as complaints about depository institutions with less than $10 billion in assets, are not published in the Consumer Complaint Database. The database generally updates daily.


▶ [data.gov/dataset/consumer-complaint-database](https://catalog.data.gov/dataset/consumer-complaint-database)

In [4]:
import pandas as pd
import functools

@functools.lru_cache(maxsize=1)
def load_data_1(url):
    dtype = {
        'Complaint ID': str,
        'Product': str,
        'Sub-product': str,
        'Issue': str,
        'Sub-issue': str,
        'State': str,
        'ZIP code': str,
        'Submitted via': str,
        'Date received': str,
        'Date sent to company': str,
        'Company': str,
        'Company response to consumer': str,
        'Timely response?': str,
        'Consumer disputed?': str,
        'Complaint narrative': str,
        'Company public response': str,
        'Tags': str,
        'Consumer consent provided?': str,
        'Company': str,
        'State': str,
        'ZIP code': str,
        'Tags': str,
        'Consumer consent provided?': str,
        'Submitted via': str,
        'Date received': str,
        'Date sent to company': str,
        'Company response to consumer': str,
        'Timely response?': str,
        'Consumer disputed?': str,
        'Complaint narrative': str,
        'Company public response': str,
    }

    df = pd.read_csv(url, dtype=dtype, compression='zip')
    return df

In [5]:
consumer_complaint_data = load_data_1('https://files.consumerfinance.gov/ccdb/complaints.csv.zip')
consumer_complaint_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2024-09-18,Credit reporting or other personal consumer re...,Other personal consumer report,Incorrect information on your report,Information belongs to someone else,NaN,NaN,ENCORE CAPITAL GROUP INC.,FL,34472,NaN,NaN,Web,2024-09-18,Closed with explanation,Yes,NaN,10169554
1,2024-09-19,Student loan,Private student loan,Struggling to repay your loan,Bankruptcy,NaN,NaN,CL Holdings LLC,KY,40160,NaN,NaN,Web,2024-09-19,In progress,Yes,NaN,10169752
2,2024-07-24,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,NaN,NaN,"EQUIFAX, INC.",GA,30052,NaN,Consent not provided,Web,2024-07-24,Closed with non-monetary relief,Yes,NaN,9604836
3,2024-07-27,Credit card,General-purpose credit card or charge card,"Advertising and marketing, including promotion...",Didn't receive advertised or promotional terms,"Hello, I got the capital one XXXX credit card ...",NaN,CAPITAL ONE FINANCIAL CORPORATION,NY,10032,NaN,Consent provided,Web,2024-07-27,Closed with explanation,Yes,NaN,9634523
4,2024-07-01,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,30253,NaN,Consent not provided,Web,2024-07-01,Closed with non-monetary relief,Yes,NaN,9390464


In [6]:
# Save dataset as CSV
consumer_complaint_data.to_csv('./content/consumer_complaint_data.csv', index=False)

#### FDIC failed bank list dataset

> The FDIC is often appointed as receiver for failed banks. This list includes banks which have failed since October 1, 2000.

▶ [data.gov/dataset/fdic-failed-bank-list](https://catalog.data.gov/dataset/fdic-failed-bank-list)


In [7]:
@functools.lru_cache(maxsize=1)
def load_data_2(url):
    try:
        df = pd.read_csv(url, encoding='utf-8')
    except pd.errors.ParserError as e:
        error_message = str(e)
        line_number = int(error_message.split("line ")[1].split(",")[0])  # Extract line number from error

        with open(url, 'r', encoding='utf-8') as file:
            for i, line in enumerate(file):
                if i + 1 == line_number:
                    print(f"Problematic line {line_number}: {line}")
                    break
        df = pd.read_csv(url, encoding='utf-8')
    return df

  # try:
  #   df = pd.read_csv(url, encoding='utf-8')
  # except UnicodeDecodeError:
  #   df = pd.read_csv(url, encoding='latin-1')
  # return df


In [8]:
!wget https://www.fdic.gov/system/files/2024-07/banklist.csv

# https://www.fdic.gov/bank/individual/failed/banklist.csv

--2024-10-10 02:41:40--  https://www.fdic.gov/system/files/2024-07/banklist.csv
Resolving www.fdic.gov (www.fdic.gov)... 23.222.16.32, 23.222.16.73, 2600:1408:ac00::17de:1049, ...
Connecting to www.fdic.gov (www.fdic.gov)|23.222.16.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45390 (44K) [text/csv]
Saving to: ‘banklist.csv.4’

banklist.csv.4      100%[===================>]  44.33K  --.-KB/s    in 0.04s   

2024-10-10 02:41:41 (1.22 MB/s) - ‘banklist.csv.4’ saved [45390/45390]



In [9]:
failed_banks = load_data_2('./banklist.csv')
failed_banks.head()

,Bank Name,City,State,Cert,Acquiring Institution,Closing Date,Fund
0,Republic First Bank dba Republic Bank,Philadelphia,PA,27332,"Fulton Bank, National Association",26-Apr-24,10546
1,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,3-Nov-23,10545
2,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.",28-Jul-23,10544
3,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.",1-May-23,10543
4,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.",12-Mar-23,10540


In [10]:
# Save dataset as CSV
failed_banks.to_csv('./content/failed_banks_data.csv', index=False)

## Query CSV file

#### Instantiate CSV agent

In [11]:
PROJECT_ID = "my-project-0004-346516"
# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name='gemini-1.5-pro-001', #"gemini-pro", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)

In [12]:
csv_agent = create_csv_agent(
    path=["./content/consumer_complaint_data.csv", "./content/failed_banks_data.csv"],
    llm=llm,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

#### Basic queries

In [20]:
# 1: How many rows are there in each dataset?
csv_agent.invoke(str(input()))

 How many rows are there in each dataset




> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'How many rows are there in each dataset', 'output': ''}

In [14]:
# Confirm answer using Pandas
print("Consumer complaint # rows: ", consumer_complaint_data.shape[0])
print("Failed banks # rows: ",failed_banks.shape[0])

Consumer complaint # rows:  6347764
Failed banks # rows:  569


In [21]:
# 1: How many rows are there in each dataset?
csv_agent.invoke(str(input()))

 How many rows are there in each dataset




> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'How many rows are there in each dataset', 'output': ''}

## What is a REPL(Read-Eval-Print Loop) tool?


> A **REPL** tool is an interactive programming environment that takes single user inputs (**R**ead), and executes them (**E**val), returns the result to the user (**P**rint), and waits for the next input (**L**oop).

▶ [Python AST Module Documentation](https://docs.python.org/3/library/ast.html)





In [16]:
import ast
import pandas as pd

class PythonAstREPLTool:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.env = {'df': dataframe}  # Execution environment with DataFrame

    def execute(self, code):
        try:
            # Parse the code into an AST
            tree = ast.parse(code, mode='exec')

            # Compile the AST into a code object
            code_obj = compile(tree, filename="<ast>", mode="exec")

            # Execute the compiled code in the environment
            exec(code_obj, self.env)

            # Capture the result
            result = self.env.get('result', None)
            return result
        except Exception as e:
            return str(e)

In [34]:
df = pd.read_csv('./content/sample_data/california_housing_test.csv')
repl_tool = PythonAstREPLTool(df)

# Code to get the number of rows
code = "result = df.shape[0]"
print(repl_tool.execute(code))  # Should print 3000 as number of rows

3000


## Using Langchain packages

In [23]:
!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv


--2024-10-10 04:40:22--  https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44225 (43K) [text/csv]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  43.19K  --.-KB/s    in 0.1s    

2024-10-10 04:40:23 (386 KB/s) - ‘titanic.csv’ saved [44225/44225]



In [24]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


In [25]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

887

In [26]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

In [27]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name='gemini-1.5-pro-002', #"gemini-1.0-pro-002", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/jupyter/.config/sagemaker/config.yaml


In [30]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm=llm, db=db,  verbose=True)

In [31]:
agent_executor.invoke({"input": "what's the average age of survivors"})





> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: titanicI should check the schema of the titanic table for age and survived information.
Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/I can query the titanic table for the average age of survivors.
Action: sql_db_query_checker
Action Input: SELECT avg(Age) FROM titanic WHERE Survived = 1```sql

{'input': "what's the average age of survivors",
 'output': 'The average age of survivors is 28.4.'}

## Creating advance code from python and later we execute it for the daaframe defined above.

In [52]:
ai_msg = llm.invoke(
    "I have a pandas DataFrame 'df' with columns 'housing_median_age' and 'households'. Write code to compute the correlation between the two columns. Return Markdown for a Python code snippet and nothing else."
)
print(ai_msg.content)

```python
import pandas as pd

# Assuming 'df' is your DataFrame
correlation = df['housing_median_age'].corr(df['households'])

print(correlation)

```



In [49]:
print(ai_msg.content[3:-4])

python
import pandas as pd

# Assuming 'df' is your pandas DataFrame with 'housing_median_age' and 'households' columns

correlation = df['housing_median_age'].corr(df['households'])
print(correlation)




In [50]:
# Code to get the number of rows
code = ai_msg.content[9:-4]
print(repl_tool.execute(code))  # Should print 3000 as number of rows

-0.30517077522973896
3000
